# Extraction

In [41]:
import pandas as pd
import requests
import mysql.connector
import csv

### Extracting from mysql locally

In [42]:
def extractMysql():
    mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="CustomerData"
    )
    cursor = mydb.cursor()
    cursor.execute("SELECT * FROM Customers")
    customers_data = cursor.fetchall()
    cursor.close()
    mydb.close()
    return customers_data


### Extracting from CSV File

In [43]:
def extractCSV():
    data = []
    with open('Order_Info.csv', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            data.append(row)
    return data

### Extracting from Locally Hosted API

In [44]:
def extractAPI():
    url = 'http://127.0.0.1:5000/data'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Error: Unable to fetch data from API")
        return []


# Transformation 

In [45]:
def tranformCustomer(customers_data):
    transformed_data = []
    for row in customers_data:
        transformed_row = {
            "customer_id": row[0],
            "customer_name": row[1],
            "segment": row[2],
            "country": row[3],
            "city": row[4],
            "state": row[5],
            "postal_code": row[6],
            "region": row[7]
        }
        transformed_data.append(transformed_row)
    return transformed_data

In [46]:
def tranformOrder(csv_data):
    transformed_data = []
    for row in csv_data:
        transformed_row = {
            "order_id": row["Order ID"],
            "order_date": row["Order Date"],
            "ship_date": row["Ship Date"],
            "ship_mode": row["Ship Mode"],
            "customer_id": row["Customer ID"],
            "sales": row["Sales"],
            "quantity": row["Quantity"],
            "discount": row["Discount"],
            "profit": row["Profit"]
        }
        transformed_data.append(transformed_row)
    return transformed_data

In [47]:
def transformProduct(api_data):
    transformed_data = []
    for row in api_data:
        transformed_row = {
            "order_id": row["Order ID"],
            "product_id": row["Product ID"],
            "product_name": row["Product Name"],
            "category": row["Category"],
            "sub_category": row["Sub-Category"],
            "price": row["Price"]
        }
        transformed_data.append(transformed_row)
    return transformed_data

##### Performing Extraction

In [48]:
customerData=extractMysql()
orderData=extractCSV()
productData=extractAPI()

##### Performing Transfomration

In [49]:
customerData_Transform=tranformCustomer(customerData)
orderData_Transform=tranformOrder(orderData)
productData_Transform=transformProduct(productData)

# Loading

In [69]:
try:
    connection = mysql.connector.connect(
        host="localhost",
        user="root",
        password="root",
        database="STORESALE"
    )
    cursor = connection.cursor()
    
    for customer in customerData_Transform:
        cursor.execute("INSERT IGNORE INTO Customers (CustomerID, CustomerName, Segment, Country, City, State, PostalCode, Region) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)", 
                   (customer['customer_id'], customer['customer_name'], customer['segment'], customer['country'], customer['city'], customer['state'], customer['postal_code'], customer['region']))

    print("Customer Done")
    for product in productData_Transform:
        cursor.execute("INSERT Ignore INTO Products (ProductID, ProductName, CategoryName, Price) VALUES (%s, %s, %s, %s)", 
                       (product['product_id'], product['product_name'], product['category'], product['price']))
    print("Products Done")   
    for order in orderData_Transform:
        cursor.execute("INSERT IGNORE INTO Orders (OrderID, CustomerID, OrderDate, ShipDate, ShipMode, Sales, Quantity, Discount, Profit) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)", 
                       (order['order_id'], order['customer_id'], order['order_date'], order['ship_date'], order['ship_mode'], order['sales'], order['quantity'], order['discount'], order['profit']))
    print("Order Done")
    
    for category in productData_Transform:
        cursor.execute("INSERT IGNORE INTO Categories (CategoryName,SubCategory) VALUES (%s, %s)", 
                       (category['category'], category['sub_category']))
    print("Category Done")
    
    orderItemData_Transform = []
    for order in orderData_Transform:
        for product in productData_Transform:
            if order['order_id'] == product['order_id']:
                orderItemData_Transform.append({'order_id': order['order_id'], 'product_id': product['product_id'], 'quantity': order['quantity']})
    for order_item in orderItemData_Transform:
        cursor.execute("INSERT IGNORE INTO Order_Items (OrderID, ProductID, Quantity) VALUES (%s, %s, %s)", 
                       (order_item['order_id'], order_item['product_id'], order_item['quantity']))
    print("Order Item Done")
    
    connection.commit()

except mysql.connector.Error as error:
    print("Failed to insert record into MySQL table:", error)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed.")


Customer Done
Products Done
Order Done
Category Done
Order Item Done
MySQL connection is closed.
